# Computing photometry and SEDs of diffsky galaxies

This notebook shows how to compute photometry through arbitrary bands of galaxies in a diffsky mock, and how to compute high-res SEDs.

## Loading the mock data

The next two cells cells show how to load the mock files that are natively produced by the diffsky source code. For purposes of this demo, we will just work with a single lightcone patch of the natively-generated hdf5 files, which will be downloaded in the next cell.

#### Using the OpenCosmo toolkit
The hdf5 files produced by diffsky are later ingested by the [OpenCosmo](https://opencosmo.readthedocs.io/en/stable/) toolkit, which enables efficient querying, map-making, and other features. See [Accessing and Working With Diffsky Mock Galaxy Catalogs](https://github.com/ArgonneCPAC/opencosmo-examples/blob/main/03-Diffsky/demo_diffmah_diffstar.ipynb) for a tutorial on how to load diffsky mocks with OpenCosmo.

In [ ]:
! wget -P ./download_dir -q -r -e robots=off -np -nH --cut-dirs=7 --reject "index.html*" https://portal.nersc.gov/project/hacc/aphearin/diffsky_data/smdpl_dr1_12_03_2025

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from diffsky.data_loaders.hacc_utils import load_lc_mock as llcm
import os
drn_mock = "download_dir"
# drn_mock = "/Users/aphearin/work/DATA/random_data/1203/smdpl_dr1"
bn_mock = "lc_cores-475.118.diffsky_gals.hdf5"

diffsky_lc_patch = llcm.load_diffsky_lc_patch(drn_mock, bn_mock)

### Inspect the mock data

The `diffsky_lc_patch` dictionary stores all data columns of the mock, along with supplementary metadata needed to compute photometry.

In [ ]:
print(diffsky_lc_patch.keys())

### Recompute mock photometry

The `diffsky_lc_patch` dictionary has all the keys expected by the `compute_phot_from_diffsky_mock` function, which we use in the next two cells to show how to recompute photometry and enforce agreement with the columns stored in the mock.

In [ ]:
phot_info = llcm.compute_phot_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
assert np.allclose(diffsky_lc_patch['diffsky_data']['lsst_i'], phot_info['obs_mags'][:, 3], rtol=1e-4)

### Recompute photometry of disk/bulge/knot decomposition

The photometry of each morphological component can be calculated using the `compute_dbk_phot_from_diffsky_mock` function.

In [ ]:
dbk_phot_info = llcm.compute_dbk_phot_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
assert np.allclose(diffsky_lc_patch['diffsky_data']['lsst_i_bulge'], dbk_phot_info['obs_mags_bulge'][:, 3], rtol=1e-4)

### Compute SED of diffsky galaxies

In [ ]:
sed_info = llcm.compute_sed_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.loglog()
xlim = ax.set_xlim(2_000, 200_000)
ylim = ax.set_ylim(1e-8, 1e-3)

igal = 50
__=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, sed_info['rest_sed'][igal, :])
xlabel = ax.set_xlabel('wavelength [angstroms]')
ylabel = ax.set_ylabel('Lsun/Hz')

### Compute SED of disk/bulge/knot components

In [ ]:
dbk_sed_info = llcm.compute_dbk_sed_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.loglog()
xlim = ax.set_xlim(2_000, 200_000)
ylim = ax.set_ylim(1e-8, 1e-4)

igal = 50
__=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_bulge'][igal, :])
__=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_disk'][igal, :])
__=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_knots'][igal, :])
xlabel = ax.set_xlabel('wavelength [angstroms]')
ylabel = ax.set_ylabel('Lsun/Hz')

### Computing photometry in other bands

You can compute photometry in other bandpasses by replacing the `tcurves` entry of `diffsky_lc_patch` with any sequence of transmission curves. 

Each individual transmission curve must be a namedtuple with two fields: `wave` and `transmission`, and your list of transmission curves also needs to be formatted as a namedtuple, using whatever names you want to name each bandpass. 

The next few cells show how to compute photometry through two fake bandpasses.

In [ ]:
from jax.scipy.stats import norm as jnorm
from collections import namedtuple
TransmissionCurve = namedtuple("TransmissionCurve", ("wave", "transmission"))

wave = np.linspace(200, 8_000, 500)

fake_tcurve1 = jnorm.pdf(wave, loc=3_000.0, scale=500.0)
fake_tcurve1 = TransmissionCurve(wave, fake_tcurve1/fake_tcurve1.max())

fake_tcurve2 = jnorm.pdf(wave, loc=5_000.0, scale=500.0)
fake_tcurve2 = TransmissionCurve(wave, fake_tcurve2/fake_tcurve2.max())

fig, ax = plt.subplots(1, 1)
__=ax.plot(fake_tcurve1.wave, fake_tcurve1.transmission)
__=ax.plot(fake_tcurve2.wave, fake_tcurve2.transmission)

In [ ]:
Tcurves = namedtuple("Tcurves", ("fake_tcurve1", "fake_tcurve2"))
fake_tcurves = Tcurves(fake_tcurve1, fake_tcurve2)

diffsky_lc_patch['tcurves'] = fake_tcurves

phot_info = llcm.compute_dbk_phot_from_diffsky_mock(**diffsky_lc_patch)

In [ ]:
phot_info['obs_mags'].shape